# 제2회 신약 개발 AI 경진대회

---

# 환경 설정

---

In [1]:
%pip install -q rdkit seaborn matplotlib scikit-learn pandas numpy tqdm deepchem mendeleev torch torch-geometric pytorch-lightning torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.4/367.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.8/865.8 kB 19.2 MB/s eta 0:00:00


In [2]:
# 코랩 - 구글 드라이브 연결
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# 추출가능 피처 사전 확인

---

## mendeleev

In [ ]:
from mendeleev.fetch import fetch_table

# 원소 특성 테이블 가져오기
elements_table = fetch_table('elements')
print(elements_table.columns)
print(f"\n피처 개수: {len(elements_table.columns)}")

Index(['annotation', 'atomic_number', 'atomic_radius', 'atomic_volume',
       'block', 'density', 'description', 'dipole_polarizability',
       'electron_affinity', 'electronic_configuration', 'evaporation_heat',
       'fusion_heat', 'group_id', 'lattice_constant', 'lattice_structure',
       'name', 'period', 'series_id', 'specific_heat_capacity', 'symbol',
       'thermal_conductivity', 'vdw_radius', 'covalent_radius_cordero',
       'covalent_radius_pyykko', 'en_pauling', 'en_allen', 'jmol_color',
       'cpk_color', 'proton_affinity', 'gas_basicity', 'heat_of_formation',
       'c6', 'covalent_radius_bragg', 'vdw_radius_bondi', 'vdw_radius_truhlar',
       'vdw_radius_rt', 'vdw_radius_batsanov', 'vdw_radius_dreiding',
       'vdw_radius_uff', 'vdw_radius_mm3', 'abundance_crust', 'abundance_sea',
       'molcas_gv_color', 'en_ghosh', 'vdw_radius_alvarez', 'c6_gb',
       'atomic_weight', 'atomic_weight_uncertainty', 'is_monoisotopic',
       'is_radioactive', 'cas', 'atomic_radiu

## RDKit Descriptors

In [ ]:
from rdkit.Chem import Descriptors
import pandas as pd

# 서술자 이름 리스트 계산
descriptor_names = [desc[0] for desc in Descriptors._descList]  # 서술자 이름 리스트

# 출력
print("모든 서술자 목록:")
print(descriptor_names)
print(f"\n피처 개수: {len(descriptor_names)}")

모든 서술자 목록:
['MaxAbsEStateIndex', 'MaxEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed', 'SPS', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'AvgIpc', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP

## RDKit Lipinski

In [ ]:
from rdkit.Chem import Lipinski

# Lipinski에서 사용할 수 있는 함수 목록
lipinski_features = [func for func in dir(Lipinski) if callable(getattr(Lipinski, func)) and not func.startswith('_')]

print("Lipinski 피처 목록:")
for feature in lipinski_features:
    print(feature)

Lipinski 피처 목록:
FractionCSP3
HeavyAtomCount
NHOHCount
NOCount
NumAliphaticCarbocycles
NumAliphaticHeterocycles
NumAliphaticRings
NumAromaticCarbocycles
NumAromaticHeterocycles
NumAromaticRings
NumHAcceptors
NumHDonors
NumHeteroatoms
NumRotatableBonds
NumSaturatedCarbocycles
NumSaturatedHeterocycles
NumSaturatedRings
RingCount


## RDKit Crippen

In [ ]:
from rdkit.Chem import Crippen

# Crippen에서 사용할 수 있는 함수 목록
crippen_features = [func for func in dir(Crippen) if callable(getattr(Crippen, func)) and not func.startswith('_')]

print("Crippen 피처 목록:")
for feature in crippen_features:
    print(feature)

Crippen 피처 목록:
MolLogP
MolMR


## RDKit rdMolDescriptors

In [ ]:
from rdkit.Chem import rdMolDescriptors

# rdMolDescriptors에서 사용할 수 있는 함수 목록
rdmol_features = [func for func in dir(rdMolDescriptors) if callable(getattr(rdMolDescriptors, func)) and not func.startswith('_')]

print("rdMolDescriptors 피처 목록:")
for feature in rdmol_features:
    print(feature)

rdMolDescriptors 피처 목록:
AtomPairsParameters
BCUT2D
CalcAUTOCORR2D
CalcAUTOCORR3D
CalcAsphericity
CalcChi0n
CalcChi0v
CalcChi1n
CalcChi1v
CalcChi2n
CalcChi2v
CalcChi3n
CalcChi3v
CalcChi4n
CalcChi4v
CalcChiNn
CalcChiNv
CalcCoulombMat
CalcCrippenDescriptors
CalcEEMcharges
CalcEccentricity
CalcExactMolWt
CalcFractionCSP3
CalcGETAWAY
CalcHallKierAlpha
CalcInertialShapeFactor
CalcKappa1
CalcKappa2
CalcKappa3
CalcLabuteASA
CalcMORSE
CalcMolFormula
CalcNPR1
CalcNPR2
CalcNumAliphaticCarbocycles
CalcNumAliphaticHeterocycles
CalcNumAliphaticRings
CalcNumAmideBonds
CalcNumAromaticCarbocycles
CalcNumAromaticHeterocycles
CalcNumAromaticRings
CalcNumAtomStereoCenters
CalcNumAtoms
CalcNumBridgeheadAtoms
CalcNumHBA
CalcNumHBD
CalcNumHeavyAtoms
CalcNumHeteroatoms
CalcNumHeterocycles
CalcNumLipinskiHBA
CalcNumLipinskiHBD
CalcNumRings
CalcNumRotatableBonds
CalcNumSaturatedCarbocycles
CalcNumSaturatedHeterocycles
CalcNumSaturatedRings
CalcNumSpiroAtoms
CalcNumUnspecifiedAtomStereoCenters
CalcOxidationNumbe

# Import & Settings

---

In [3]:
import os
import random
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm

from rdkit import Chem, DataStructs
from rdkit.Chem import (
    Descriptors,
    Lipinski,
    Crippen,
    rdMolDescriptors,
    AllChem,
    MolFromSmiles
)
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from mendeleev.fetch import fetch_table
from torch import Tensor
from torch_geometric.utils.sparse import dense_to_sparse
# from transformers import AutoTokenizer
# from deepchem import feat

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# 난수 설정
def seed_everything(seed):
    """ SEED로부터 난수 생성 """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(seed=42)

In [5]:
# 데이터셋 경로 설정
ROOT_DIR_GOOGLEDRIVE = '/content/drive/MyDrive/Contest/New-Medinine-Dev/'
ROOT_DIR_LOCAL = '..'

In [ ]:
# 시각화 모듈 관련 설정
warnings.filterwarnings('ignore')           # 출력 창 경고 메시지 무시
plt.rcParams['axes.unicode_minus'] = False  # 음수부호 깨짐 관련
plt.rcParams['font.family'] = 'AppleGothic' # mac os 전용 한글 폰트 설정
# plt.rcParams['font.family'] = 'Malgun Gothic' # window 전용 한글 폰트 설정

# Load Data

---

In [6]:
class ChemicalFeatureGenerator:
    def __init__(self):
        # Mendeleev를 사용하여 추출할 원자 피처 목록
        mendeleev_atomic_feature = [
            'atomic_number',             # 원소의 원자 번호
            'atomic_radius',             # 원자의 반지름 (비결합 상태에서 측정)
            'atomic_volume',             # 원자 하나의 부피 (cm³/mol)
            'density',                   # 원소의 밀도 (g/cm³)
            'dipole_polarizability',     # 원자의 쌍극자 편극도 (외부 전기장에 유도된 전기적 변화)
            'electron_affinity',         # 전자 친화도 (전자를 얻을 때 방출되는 에너지)
            # 'electronic_configuration',  # 원자의 전자 배치 (예: 1s² 2s² 2p⁶), 문자열
            'fusion_heat',               # 융해열 (원소가 녹을 때 필요한 에너지, kJ/mol)
            'lattice_constant',          # 원소의 결정 격자 상수
            'thermal_conductivity',      # 열전도도 (W/m·K, 열 전달 능력)
            'vdw_radius',                # 반데르발스 반지름 (원자의 유효 크기)
            'covalent_radius_cordero',   # Cordero 연구 기반 공유 결합 반지름
            'covalent_radius_pyykko',    # Pyykko 연구 기반 공유 결합 반지름
            'en_pauling',                # 폴링 척도에서의 전기 음성도
            'en_allen',                  # 앨런 척도에서의 전기 음성도
            'proton_affinity',           # 양성자 친화성 (양성자와 결합할 때 방출되는 에너지)
            'gas_basicity',              # 기체 상태에서의 산염기 반응 에너지
            'heat_of_formation',         # 형성열 (원소 형성 시 방출/흡수되는 엔탈피 변화)
            'c6',                        # 분산력 상수 (van der Waals 힘)
            'vdw_radius_bondi',          # Bondi 연구 기반 반데르발스 반지름
            'vdw_radius_truhlar',        # Truhlar 연구 기반 반데르발스 반지름
            'vdw_radius_rt',             # RT 연구 기반 반데르발스 반지름
            'vdw_radius_batsanov',       # Batsanov 연구 기반 반데르발스 반지름
            'vdw_radius_dreiding',       # Dreiding 연구 기반 반데르발스 반지름
            'vdw_radius_uff',            # UFF (Universal Force Field) 모델에서의 반데르발스 반지름
            'vdw_radius_mm3',            # MM3 (Molecular Mechanics) 모델에서의 반데르발스 반지름
            'abundance_crust',           # 지각에서의 원소의 풍부도 (ppm)
            'abundance_sea',             # 바다에서의 원소의 풍부도 (ppm)
            'en_ghosh',                  # Ghosh 척도에서의 전기 음성도
            'vdw_radius_alvarez',        # Alvarez 연구 기반 반데르발스 반지름
            'c6_gb',                     # Grimme-Banner 연구 기반 분산력 상수
            'atomic_weight',             # 원자의 평균 질량
            'atomic_radius_rahm',        # Rahm 연구 기반 원자 반지름
            'covalent_radius_pyykko_double', # Pyykko 연구 기반 이중 결합 공유 결합 반지름
            'covalent_radius_pyykko_triple', # Pyykko 연구 기반 삼중 결합 공유 결합 반지름
            'molar_heat_capacity'        # 몰당 열용량 (J/mol·K)
        ]

        # 원자 피처 테이블 가져오기
        self.mendeleev_atomic_feature_table = fetch_table('elements')[mendeleev_atomic_feature]

        # # DeepChem의 Featurizer
        # self.DMPNNFeaturizer = feat.DMPNNFeaturizer()
        # self.Mol2VecFingerprint = feat.Mol2VecFingerprint()
        # self.BPSymmetryFunctionInput = feat.BPSymmetryFunctionInput(max_atoms=150)

        # # ChemBERTa 토크나이저 설정
        # archive = ['seyonec/PubChem10M_SMILES_BPE_450k', "DeepChem/ChemBERTa-77M-MTR", 'seyonec/ChemBERTa-zinc-base-v1', 'seyonec/ChemBERTa_zinc250k_v2_40k']
        # chosen = archive[2]
        # self.tokenizer = AutoTokenizer.from_pretrained(chosen)

    def _get_atomic_features(self, atom):
        """ Mendeleev 및 RDKit 기반 각 원자와 관련된 피처 추출 """
        atomic_num = atom.GetAtomicNum() - 1  # 원자 번호 추출, -1은 인덱스 보정을 위한 값

        # mendeleev 기반 원자 관련 피처 추출
        mendel_atom_f = self.mendeleev_atomic_feature_table.loc[atomic_num].to_numpy().astype(np.float32)

        # rdkit 기반 원자 관련 피처 추출
        rdkit_atom_f = [
            atom.GetDegree(),             # 원자가 결합한 다른 원자의 수 (결합 차수 무시)
            atom.GetTotalDegree(),        # 원자가 결합한 총 원자 수 (결합 차수 포함)
            atom.GetFormalCharge(),       # 원자의 형식적 전하
            atom.GetIsAromatic() * 1.,    # 원자가 방향족성인지 여부 (True/False를 1 또는 0으로 변환)
            atom.GetNumImplicitHs(),      # 원자에 연결된 암시적 수소 원자의 수
            atom.GetNumExplicitHs(),      # 원자에 연결된 명시적 수소 원자의 수
            atom.GetTotalNumHs(),         # 원자에 연결된 총 수소 원자의 수 (암시적 + 명시적)
            atom.GetNumRadicalElectrons(),# 원자의 라디칼 전자 수
            atom.GetImplicitValence(),    # 암시적 원자가 (암시적 결합을 포함한 원자의 총 원자가 수)
            atom.GetExplicitValence(),    # 명시적 원자가 (명시적으로 표시된 결합을 포함한 원자의 총 원자가 수)
            atom.GetTotalValence(),       # 총 원자가 (암시적 및 명시적 결합을 모두 포함한 원자의 총 원자가 수)
            atom.IsInRing() * 1.          # 원자가 고리 구조에 포함되었는지 여부 (True/False를 1 또는 0으로 변환)
        ]
        return mendel_atom_f, rdkit_atom_f

    def get_node_features(self, mol):
        """ 분자의 각 원자(노드)에 대한 피처를 추출, 모든 원자에 대해 피처 벡터를 생성한 후, 이를 배열로 변환하여 반환 """
        node_features = []
        for atom in mol.GetAtoms():
            mendel_atom_f, rdkit_atom_f = self._get_atomic_features(atom)
            node_features.append(np.concatenate([mendel_atom_f, rdkit_atom_f])) # 두 종류의 피처를 배열로 결합
        return np.array(node_features)

    def get_edge_features(self, mol):
        """ 분자의 각 결합(엣지)에 대한 피처를 추출, 모든 결합에 대해 이러한 피처를 추출한 후, 결합에 대한 피처 배열을 반환 """
        edge_features = []
        for bond in mol.GetBonds():
            bond_features = [
                bond.GetBondTypeAsDouble(),  # 결합 유형 (단일, 이중, 삼중)
                bond.GetIsConjugated() * 1,  # 공명 여부
                bond.IsInRing() * 1          # 고리 포함 여부
            ]
            edge_features.append(bond_features)
        return np.array(edge_features)

    def get_adj_matrix(self, smiles):
        """SMILES로부터 인접 행렬과 엣지 피처 추출"""
        mol = Chem.MolFromSmiles(smiles)
        adj = Chem.rdmolops.GetAdjacencyMatrix(mol)  # 분자의 인접 행렬을 추출
        edge_index, edge_attr = dense_to_sparse(torch.tensor(adj))  # 인접 행렬을 PyTorch의 sparse 형식으로 변환
        return edge_index

    def get_molecular_features(self, mol):
        """ 분자 서술자와 관련된 피처 추출 """
        features = [
            # 분자량 관련 서술자
            Descriptors.MolWt(mol),                        # 분자량
            Descriptors.ExactMolWt(mol),                   # 정확한 분자량
            Descriptors.HeavyAtomMolWt(mol),               # 무거운 원자의 분자량

            # 극성 표면적 (TPSA)
            Descriptors.TPSA(mol),                         # 극성 표면적

            # LogP 및 지용성 관련 서술자
            Descriptors.MolLogP(mol),                      # LogP 값 (분자의 지용성)
            Crippen.MolLogP(mol),                          # Crippen 방식으로 계산된 LogP
            Descriptors.SlogP_VSA1(mol),                   # LogP와 표면적의 조합
            Descriptors.SlogP_VSA2(mol),
            Descriptors.SlogP_VSA3(mol),
            Descriptors.SlogP_VSA4(mol),
            Descriptors.SlogP_VSA5(mol),
            Descriptors.SlogP_VSA6(mol),
            Descriptors.SlogP_VSA7(mol),
            Descriptors.SlogP_VSA8(mol),
            Descriptors.SlogP_VSA9(mol),
            Descriptors.SlogP_VSA10(mol),

            # Lipinski 관련 서술자
            Lipinski.FractionCSP3(mol),                    # sp3 혼성화 탄소의 비율
            Lipinski.HeavyAtomCount(mol),                  # 무거운 원자의 개수
            Lipinski.NHOHCount(mol),                       # 수소 결합 주개의 수
            Lipinski.NOCount(mol),                         # 수소 결합 수용체의 수
            Lipinski.NumRotatableBonds(mol),               # 회전 가능한 결합의 수
            Lipinski.NumHAcceptors(mol),                   # 수소 결합 수용체의 수
            Lipinski.NumHDonors(mol),                      # 수소 결합 주개의 수
            Lipinski.NumHeteroatoms(mol),                  # 이종 원자의 수

            # E-State 서술자
            Descriptors.MaxAbsEStateIndex(mol),            # 최대 절대 E-State 지수
            Descriptors.MinAbsEStateIndex(mol),            # 최소 절대 E-State 지수
            Descriptors.MaxEStateIndex(mol),               # 최대 E-State 지수
            Descriptors.MinEStateIndex(mol),               # 최소 E-State 지수

            # BCUT 서술자
            Descriptors.BCUT2D_MWHI(mol),                  # 높은 분자량의 BCUT 값
            Descriptors.BCUT2D_MWLOW(mol),                 # 낮은 분자량의 BCUT 값
            Descriptors.BCUT2D_LOGPHI(mol),                # 높은 LogP의 BCUT 값
            Descriptors.BCUT2D_LOGPLOW(mol),               # 낮은 LogP의 BCUT 값

            # 분자 기하학적 서술자
            Descriptors.BalabanJ(mol),                     # Balaban J 지수
            Descriptors.BertzCT(mol),                      # 분자의 복잡도

            # 연결성 서술자
            Descriptors.Chi0n(mol),                        # 0차 Chi 지수 (비선형)
            Descriptors.Chi1n(mol),                        # 1차 Chi 지수 (비선형)
            Descriptors.Chi2n(mol),                        # 2차 Chi 지수 (비선형)

            # 분자 표면적
            Descriptors.LabuteASA(mol),                    # 접근 가능한 표면적

            # Crippen 서술자
            Crippen.MolMR(mol),                            # Crippen 방식으로 계산된 몰 굴절률

            # 고리 관련 서술자
            Lipinski.NumAliphaticCarbocycles(mol),         # 지방족 탄소 고리 수
            Lipinski.NumAliphaticHeterocycles(mol),        # 지방족 이종 고리 수
            Lipinski.NumAromaticCarbocycles(mol),          # 방향족 탄소 고리 수
            Lipinski.NumAromaticRings(mol),                # 방향족 고리 수
            Lipinski.RingCount(mol),                       # 고리의 총 개수
            Lipinski.NumSaturatedCarbocycles(mol),         # 포화된 탄소 고리 수
            Lipinski.NumSaturatedHeterocycles(mol),        # 포화된 이종 고리 수
            Lipinski.NumSaturatedRings(mol),               # 포화된 고리 수

            # 기타 분자 특성
            rdMolDescriptors.CalcNumAtoms(mol),            # 분자의 원자 수
            rdMolDescriptors.CalcNumHeavyAtoms(mol),       # 무거운 원자의 수
        ]

        # 특정 서브구조 매칭
        num_carboxyl_groups = len(mol.GetSubstructMatches(MolFromSmiles("[C](=O)[OH]"))) # 카르복실 그룹 수
        num_amino_groups = len(mol.GetSubstructMatches(MolFromSmiles("[NH2]")))          # 아미노 그룹 수
        num_ammonium_groups = len(mol.GetSubstructMatches(MolFromSmiles("[NH4+]")))      # 암모늄 그룹 수
        num_sulfonic_acid_groups = len(mol.GetSubstructMatches(MolFromSmiles("[S](=O)(=O)[O-]")))  # 설폰산 그룹 수
        num_alkoxy_groups = len(mol.GetSubstructMatches(MolFromSmiles('CO')))            # 알콕시 그룹 수

        features.extend([num_carboxyl_groups, num_amino_groups, num_ammonium_groups, num_sulfonic_acid_groups, num_alkoxy_groups])
        return np.array(features)

    def get_fingerprints(self, mol):
        """ 분자 지문(fingerprint)과 관련된 피처 추출 """
        # MorganGenerator로 반지름 6 및 반지름 3의 Morgan 지문 생성
        generator_12 = GetMorganGenerator(radius=6, fpSize=2048) # 1024, 반경 2
        # generator_6 = GetMorganGenerator(radius=3, fpSize=2048)

        ecfp12 = generator_12.GetFingerprint(mol)  # 반지름 6으로 생성된 ECFP 지문 (2048 비트 벡터)
        # ecfp6 = generator_6.GetFingerprint(mol)   # 반지름 3으로 생성된 ECFP 지문 (2048 비트 벡터)
        # 기타 지문 생성
        maccs = Chem.rdMolDescriptors.GetMACCSKeysFingerprint(mol)          # 166개의 이진 피처로 구성된 MACCS Keys 지문
        rdk_fp = Chem.RDKFingerprint(mol)                                   # 기본적인 RDK 지문. 분자의 연결성을 기반으로 특징을 추출
        layer_fp = Chem.rdmolops.LayeredFingerprint(mol)                    # 레이어드 지문으로, 원자의 특성 및 결합 정보를 단계별로 추가하여 지문을 생성
        pattern_fp = Chem.rdmolops.PatternFingerprint(mol)                  # 패턴 기반 지문으로, 원자 간 결합 패턴을 반영한 피처를 추출

        # 각 지문을 numpy 배열로 변환
        ecfp12_np = np.zeros((1,), dtype=np.int8)
        # ecfp6_np = np.zeros((1,), dtype=np.int8)
        maccs_np = np.zeros((1,), dtype=np.int8)
        rdk_fp_np = np.zeros((1,), dtype=np.int8)
        layer_fp_np = np.zeros((1,), dtype=np.int8)
        pattern_fp_np = np.zeros((1,), dtype=np.int8)

        DataStructs.ConvertToNumpyArray(ecfp12, ecfp12_np)
        # DataStructs.ConvertToNumpyArray(ecfp6, ecfp6_np)
        DataStructs.ConvertToNumpyArray(maccs, maccs_np)
        DataStructs.ConvertToNumpyArray(rdk_fp, rdk_fp_np)
        DataStructs.ConvertToNumpyArray(layer_fp, layer_fp_np)
        DataStructs.ConvertToNumpyArray(pattern_fp, pattern_fp_np)

        # 모든 지문을 하나의 numpy 배열로 결합하여 반환
        # ecfp6_np,
        return np.hstack([ecfp12_np, maccs_np, rdk_fp_np, layer_fp_np, pattern_fp_np])

    # def get_deepchem_features(self, smiles):
    #     """ DeepChem 기반 피처 추출 """
    #     mol2vec_features = self.Mol2VecFingerprint(smiles) # 분자의 SMILES 문자열을 입력으로 받아, 단어 임베딩과 유사하게 분자를 벡터화한 피처를 추출
    #     dmpnn_features = self.DMPNNFeaturizer.featurize([smiles])[0] # 분자의 그래프 구조를 기반으로 노드(원자) 피처를 추출
    #     return np.concatenate([mol2vec_features, dmpnn_features.node_features], axis=1)

    # def encode_smiles(self, smiles):
    #     """ SMILES 문자열을 토큰화 """
    #     inputs = self.tokenizer.encode_plus(smiles, padding=True, return_tensors='pt')
    #     return inputs['input_ids']

    def get_adj_matrix(self, smiles):
        """ SMILES로부터 인접 행렬과 엣지 피처 추출 """
        mol = MolFromSmiles(smiles)
        adj = Chem.rdmolops.GetAdjacencyMatrix(mol)
        edge_index, edge_attr = dense_to_sparse(Tensor(adj))
        return edge_index, edge_attr

    def extract_all_features(self, smiles):
        """ SMILES을 기반으로 다양한 피처 추출 """
        mol = MolFromSmiles(smiles)

        # 1. 노드 피처
        # node_features = self.get_node_features(mol)

        # 2. 엣지 피처
        # edge_features = self.get_edge_features(mol)

        # 3. 분자 서술자
        molecular_features = self.get_molecular_features(mol)

        #################################################

        # 4. 지문 (fingerprint)
        # fingerprints = self.get_fingerprints(mol)

        # # 5. DeepChem 기반 피처
        # deepchem_features = self.get_deepchem_features(smiles)

        # # 6. SMILES 시퀀스 기반 토큰
        # smiles_token = self.encode_smiles(smiles)

        return {
            # 'node_features': node_features,
            # 'edge_features': edge_features,
            'edge_index': self.get_adj_matrix(smiles),
            'molecular_features': molecular_features,
            # 'fingerprints': fingerprints,
            # 'deepchem_features': deepchem_features,
            # 'smiles_token': smiles_token
        }


In [7]:
# 엣지 인덱스를 생성하는 함수
def generate_edge_indices(smiles_list):
    generator = ChemicalFeatureGenerator()
    edge_indices = []
    for smiles in smiles_list:
        edge_index = generator.get_adj_matrix(smiles)
        edge_indices.append(edge_index)
    return edge_indices

In [34]:
def process(df, generator):
    """ SMILES 데이터를 기반으로 피처 추출 """
    all_features = []
    edge_indices = []

    for sample in tqdm(df.Smiles):
        features = generator.extract_all_features(sample)

        # 피처를 결합하여 하나의 벡터로 만듦
        combined_features = np.concatenate([
                # features['node_features'].flatten(),
                # features['edge_features'].flatten(),
                features['molecular_features'],
                # features['fingerprints']
        ])

        all_features.append(combined_features)

        # edge_index가 None이 아닌지 체크하고 추가
        if features['edge_index'] is not None:
            edge_indices.append(features['edge_index'])
        else:
            # edge_index가 None인 경우 기본 값을 설정하거나 로그를 남기기
            print(f"SMILES: {sample}에서 edge_index가 생성되지 않았습니다.")
            edge_indices.append(torch.zeros((2, 0), dtype=torch.long))  # 예시로 빈 edge_index 추가

    # 추출된 모든 피처를 DataFrame으로 변환
    features_df = pd.DataFrame(all_features)

    # 기존 df와 추출된 피처들을 병합 (기존 컬럼 유지)
    new_df = pd.concat([df.reset_index(drop=True), features_df], axis=1)

    # edge_indices 리스트를 하나의 큰 텐서로 변환
    # edge_indices가 여러 개의 (2, num_edges) 형식이므로 이를 텐서로 변환
    edge_indices = [torch.tensor(e, dtype=torch.long) for e in edge_indices]

    return new_df, edge_indices

In [32]:
# if __name__ == '__main__':
# 데이터 로드
train = pd.read_csv(f'{ROOT_DIR_GOOGLEDRIVE}/data/train.csv')
test  = pd.read_csv(f'{ROOT_DIR_GOOGLEDRIVE}/data/test.csv')

# IC50 값을 따로 저장 (IC50_nM 열을 따로 빼둠)
train_ic50 = train['IC50_nM'].copy()

# train 데이터에서 'SMILES'와 'pIC50' 열만 남기고 나머지 삭제
train = train[['Smiles', 'pIC50']]

In [35]:
# 새로운 피처 추가
generator = ChemicalFeatureGenerator()
trainset, edge_indices_train = process(train, generator)
testset, edge_indices_test = process(test, generator)




100%|██████████| 1952/1952 [00:20<00:00, 95.70it/s]


TypeError: only integer tensors of a single element can be converted to an index

In [11]:
display(trainset.head())
display(testset.head())

,Smiles,pIC50,0,1,2,3,4,5,6,7,...,44,45,46,47,48,49,50,51,52,53
0,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...,10.66,995.188,994.516405,924.628,250.87,2.74360,2.74360,41.791336,142.486570,...,1.0,2.0,3.0,142.0,72.0,0.0,0.0,0.0,0.0,10.0
1,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,10.59,535.624,535.270716,501.352,106.31,2.72090,2.72090,10.216698,91.857114,...,0.0,2.0,2.0,73.0,39.0,0.0,0.0,0.0,0.0,1.0
2,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,10.11,537.596,537.249981,505.340,115.54,1.56720,1.56720,10.216698,105.070878,...,0.0,2.0,2.0,71.0,39.0,0.0,0.0,0.0,0.0,3.0
3,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,10.09,545.566,545.236222,515.326,106.31,2.43350,2.43350,10.216698,98.785474,...,0.0,1.0,1.0,69.0,39.0,0.0,0.0,0.0,0.0,1.0
4,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...,10.00,936.189,935.461533,870.669,215.17,6.49322,6.49322,31.157759,94.001218,...,0.0,2.0,2.0,132.0,67.0,0.0,0.0,0.0,0.0,5.0


,ID,Smiles,0,1,2,3,4,5,6,7,...,44,45,46,47,48,49,50,51,52,53
0,TEST_000,O=C(C1=CSC(C2=CC=CN=C2)=N1)NC3=CC(NC4CCN(C)CC4...,477.634,477.219846,446.386,79.38,5.28490,5.28490,15.370440,53.058039,...,1.0,1.0,2.0,65.0,34.0,0.0,0.0,0.0,0.0,2.0
1,TEST_001,N#CC1=CC(C=C2)=C(C=C1)N2C(N=C3)=NC(NC4CCCCC4)=...,466.593,466.259343,436.353,96.38,4.82478,4.82478,10.633577,43.446895,...,1.0,1.0,2.0,65.0,35.0,0.0,0.0,0.0,0.0,0.0
2,TEST_002,N#CC(C=C1)=C(N[C@@H]2CCNC2)C=C1NC(N=C3)=NC=C3C...,415.505,415.223292,390.305,115.51,2.26358,2.26358,21.267154,51.969308,...,0.0,2.0,2.0,56.0,31.0,0.0,0.0,0.0,0.0,0.0
3,TEST_003,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(NC(C=C)=O)CC...,498.591,498.249172,468.351,129.78,3.94208,3.94208,15.950366,50.953067,...,1.0,1.0,2.0,67.0,37.0,0.0,0.0,0.0,0.0,2.0
4,TEST_004,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(N)CC3)=C2C(C...,444.543,444.238608,416.319,126.70,3.59858,3.59858,16.367245,45.045887,...,1.0,1.0,2.0,61.0,33.0,0.0,0.0,0.0,0.0,2.0


In [12]:
# NaN 비율 계산
nan_ratio = trainset.isna().mean()

# NaN 비율이 0.1 (10%) 이상인 피처들만 추출
nan_columns_above = nan_ratio[nan_ratio > 0]

# 10% 이상 NaN 값을 가진 피처의 개수 출력
print(f"NaN 값이 1개 이상인 피처의 개수: {len(nan_columns_above)}")

# 해당 피처들의 이름과 비율 출력
print(nan_columns_above)

NaN 값이 1개 이상인 피처의 개수: 0
Series([], dtype: float64)


In [14]:
# 피처를 CSV로 저장
trainset.to_csv(f'{ROOT_DIR_GOOGLEDRIVE}/preprocessed_data/trainset1.csv', index=False)
testset.to_csv(f'{ROOT_DIR_GOOGLEDRIVE}/preprocessed_data/testset1.csv', index=False)

# edge_index는 numpy 파일로 저장 (Pickle 사용)
# np.save(f'{ROOT_DIR_GOOGLEDRIVE}/preprocessed_data/edge_indices_train.npy', edge_indices_train)
# np.save(f'{ROOT_DIR_GOOGLEDRIVE}/preprocessed_data/edge_indices_test.npy', edge_indices_test)

# Import

In [15]:
import numpy as np

import torch
import torch.nn.functional as F
from torch_geometric.utils import to_dense_adj
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader
# from pytorch_lightning import LightningModule, LightningDataModule
import pytorch_lightning as pl

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [16]:
CFG = {
    'input_dim': 128,  # 예시로 SMILES로부터 추출한 피처의 크기 (입력 크기)
    'hidden_dim': 64,  # 은닉층 크기
    'output_dim': 1,   # IC50 예측 회귀 문제이므로 출력은 1차원
    'num_layers': 3,   # GCN 레이어 수
    'dropout': 0.3,    # 드롭아웃 비율
    'lr': 0.001,       # 학습률
    'batch_size': 32,  # 배치 크기
    'epochs': 50       # 에폭 수
}

In [27]:
class GCNModel(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
        super().__init__()
        self.layers = torch.nn.ModuleList()
        self.layers.append(GCNConv(input_dim, hidden_dim))
        for _ in range(num_layers - 2):
            self.layers.append(GCNConv(hidden_dim, hidden_dim))
        self.layers.append(GCNConv(hidden_dim, output_dim))
        self.dropout = dropout

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for layer in self.layers[:-1]:
            x = layer(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.layers[-1](x, edge_index)
        return x

    def training_step(self, batch, batch_idx):
        out = self(batch).squeeze()
        loss = F.mse_loss(out, batch.y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        out = self(batch).squeeze()
        loss = F.mse_loss(out, batch.y)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        out = self(batch).squeeze()
        loss = F.mse_loss(out, batch.y)
        self.log('test_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=CFG['lr'])


class GNNDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, val_dataset, test_dataset, batch_size):
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset
        self.batch_size = batch_size

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)


from torch_geometric.data import Dataset

class GraphDataset(Dataset):
    def __init__(self, x_data, y_data, edge_indices):
        self.x_data = x_data
        self.y_data = y_data
        self.edge_indices = edge_indices

    def len(self):
        return len(self.x_data)

    def get(self, idx):
        x = torch.tensor(self.x_data[idx], dtype=torch.float)
        y = torch.tensor(self.y_data[idx], dtype=torch.float)
        edge_index = self.edge_indices[idx]  # 각 sample마다 해당하는 edge_index를 가져옴
        return Data(x=x, y=y, edge_index=edge_index)


In [28]:
# 데이터셋 분리 및 모델 학습 파이프라인
def prepare_data(trainset, testset, edge_indices_train, edge_indices_test):
    train_x = trainset.drop(columns=['pIC50', 'Smiles']).values  # pIC50와 Smiles를 제외한 나머지 피처들로 학습 데이터셋 생성
    train_y = trainset['pIC50'].values                           # pIC50 값을 numpy 배열로 변환
    test_x = testset.drop(columns=['Smiles', 'ID']).values       # Smiles, ID 제외한 나머지 피처들로 테스트 데이터셋 생성

    # 데이터셋을 학습, 검증, 테스트로 나누기
    train_x, val_x, train_y, val_y, edge_indices_train, edge_indices_val = train_test_split(
        train_x, train_y, edge_indices_train, test_size=0.3, random_state=42)

    print(f"train_x shape: {train_x.shape}, train_y shape: {train_y.shape}")
    print(f"val_x shape: {val_x.shape}, val_y shape: {val_y.shape}")

    # torch_geometric용으로 Data 객체 생성 (샘플별로)
    train_dataset = GraphDataset(train_x, train_y, edge_indices_train)
    val_dataset = GraphDataset(val_x, val_y, edge_indices_val)
    test_dataset = GraphDataset(test_x, np.zeros(test_x.shape[0]), edge_indices_test)  # test 데이터에는 y가 없으므로 0으로 대체

    return train_dataset, val_dataset, test_dataset

In [19]:
# RMSE 계산 함수
def pIC50_to_IC50(pic50_values):
    """Convert pIC50 values to IC50 (nM)."""
    return 10 ** (9 - pic50_values)

def evaluate_model(model, datamodule):
    """RMSE 평가 함수"""
    val_loader = datamodule.val_dataloader()
    preds, true_vals = [], []

    for batch in val_loader:
        with torch.no_grad():
            pred = model(batch).squeeze()
            preds.append(pred.cpu().numpy())
            true_vals.append(batch.y.cpu().numpy())

    preds = np.concatenate(preds)
    true_vals = np.concatenate(true_vals)

    # pIC50을 IC50으로 변환하여 MSE 및 RMSE 계산
    mse = mean_squared_error(pIC50_to_IC50(true_vals), pIC50_to_IC50(preds))
    rmse = np.sqrt(mse)

    return rmse

In [29]:
# 학습 및 추론
# if __name__ == '__main__':

# 데이터 로드 및 edge_index 불러오기
trainset = pd.read_csv(f'{ROOT_DIR_GOOGLEDRIVE}/preprocessed_data/trainset1.csv')
testset = pd.read_csv(f'{ROOT_DIR_GOOGLEDRIVE}/preprocessed_data/testset1.csv')

# 저장된 edge_index 불러오기 (예시로 numpy 저장 및 불러오는 형식 사용)
# edge_indices_train = np.load(f'{ROOT_DIR_GOOGLEDRIVE}/preprocessed_data/edge_indices_train.npy', allow_pickle=True)
# edge_indices_test = np.load(f'{ROOT_DIR_GOOGLEDRIVE}/preprocessed_data/edge_indices_test.npy', allow_pickle=True)


display(trainset)
display(testset)

,Smiles,pIC50,0,1,2,3,4,5,6,7,...,44,45,46,47,48,49,50,51,52,53
0,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...,10.66,995.188,994.516405,924.628,250.87,2.74360,2.74360,41.791336,142.486570,...,1.0,2.0,3.0,142.0,72.0,0.0,0.0,0.0,0.0,10.0
1,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,10.59,535.624,535.270716,501.352,106.31,2.72090,2.72090,10.216698,91.857114,...,0.0,2.0,2.0,73.0,39.0,0.0,0.0,0.0,0.0,1.0
2,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,10.11,537.596,537.249981,505.340,115.54,1.56720,1.56720,10.216698,105.070878,...,0.0,2.0,2.0,71.0,39.0,0.0,0.0,0.0,0.0,3.0
3,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,10.09,545.566,545.236222,515.326,106.31,2.43350,2.43350,10.216698,98.785474,...,0.0,1.0,1.0,69.0,39.0,0.0,0.0,0.0,0.0,1.0
4,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...,10.00,936.189,935.461533,870.669,215.17,6.49322,6.49322,31.157759,94.001218,...,0.0,2.0,2.0,132.0,67.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,O=C(Nc1nc2cc[nH]cc-2n1)c1cccc([N+](=O)[O-])c1,4.52,283.247,283.070539,274.175,113.81,2.07000,2.07000,5.316789,25.782426,...,0.0,0.0,0.0,30.0,21.0,0.0,0.0,1.0,0.0,0.0
1948,CCCCn1c(NC(=O)c2cccc(Cl)c2)nc2ccccc21,4.52,327.815,327.113840,309.671,46.92,4.74210,4.74210,5.316789,15.458258,...,0.0,0.0,0.0,41.0,23.0,0.0,0.0,0.0,0.0,0.0
1949,O=C(Nc1nc2cc(F)c(F)cc2[nH]1)c1cccc([N+](=O)[O-...,4.52,318.239,318.056447,310.175,100.92,3.00160,3.00160,5.316789,20.798448,...,0.0,0.0,0.0,31.0,23.0,0.0,0.0,1.0,0.0,0.0
1950,OC[C@H]1C[C@@H](Nc2nc(Nc3ccccc3)ncc2-c2nc3cccc...,4.38,449.536,449.152161,426.352,123.42,3.01140,3.01140,10.633577,55.128173,...,1.0,0.0,1.0,55.0,32.0,0.0,0.0,0.0,0.0,3.0


,ID,Smiles,0,1,2,3,4,5,6,7,...,44,45,46,47,48,49,50,51,52,53
0,TEST_000,O=C(C1=CSC(C2=CC=CN=C2)=N1)NC3=CC(NC4CCN(C)CC4...,477.634,477.219846,446.386,79.38,5.28490,5.28490,15.370440,53.058039,...,1.0,1.0,2.0,65.0,34.0,0.0,0.0,0.0,0.0,2.0
1,TEST_001,N#CC1=CC(C=C2)=C(C=C1)N2C(N=C3)=NC(NC4CCCCC4)=...,466.593,466.259343,436.353,96.38,4.82478,4.82478,10.633577,43.446895,...,1.0,1.0,2.0,65.0,35.0,0.0,0.0,0.0,0.0,0.0
2,TEST_002,N#CC(C=C1)=C(N[C@@H]2CCNC2)C=C1NC(N=C3)=NC=C3C...,415.505,415.223292,390.305,115.51,2.26358,2.26358,21.267154,51.969308,...,0.0,2.0,2.0,56.0,31.0,0.0,0.0,0.0,0.0,0.0
3,TEST_003,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(NC(C=C)=O)CC...,498.591,498.249172,468.351,129.78,3.94208,3.94208,15.950366,50.953067,...,1.0,1.0,2.0,67.0,37.0,0.0,0.0,0.0,0.0,2.0
4,TEST_004,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(N)CC3)=C2C(C...,444.543,444.238608,416.319,126.70,3.59858,3.59858,16.367245,45.045887,...,1.0,1.0,2.0,61.0,33.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,TEST_108,N#CC1=CC(C=C2)=C(C=C1)N2C(N=C3)=NC(N4CCOCC4)=C...,454.538,454.222957,428.330,96.82,2.91678,2.91678,10.216698,63.708331,...,0.0,2.0,2.0,60.0,34.0,0.0,0.0,0.0,0.0,2.0
109,TEST_109,O=C(C1=CSC(C2=CC=NC=C2)=N1)NC3=CC(NC4CCN(C(C)C...,561.756,561.288594,522.444,102.49,4.84700,4.84700,20.850276,70.986774,...,0.0,2.0,2.0,79.0,40.0,0.0,0.0,0.0,0.0,0.0
110,TEST_110,N#Cc1ccc(Nc2ncc(cn2)c3cnn(c3)C4CCNCC4)cc1N[C@@...,429.532,429.238942,402.316,115.51,2.65368,2.65368,21.267154,51.969308,...,0.0,2.0,2.0,59.0,32.0,0.0,0.0,0.0,0.0,0.0
111,TEST_111,O=C(C)N(CC1)CCC1N2N=CC(C3=CN=C(N4C(C=CC(C#N)=C...,411.469,411.180758,390.301,92.63,3.33908,3.33908,0.000000,48.212144,...,0.0,1.0,1.0,52.0,31.0,0.0,0.0,0.0,0.0,0.0


In [21]:
print(trainset.isna().sum())  # NaN 값이 있는지 확인
print(testset.isna().sum())  # NaN 값이 있는지 확인

Smiles    0
pIC50     0
0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
30        0
31        0
32        0
33        0
34        0
35        0
36        0
37        0
38        0
39        0
40        0
41        0
42        0
43        0
44        0
45        0
46        0
47        0
48        0
49        0
50        0
51        0
52        0
53        0
dtype: int64
ID        0
Smiles    0
0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24 

In [30]:
# edge_index는 2행 n열이어야 함 (각 열은 연결된 노드 쌍을 나타냄)
print(edge_indices_train.shape)  # (2, n) 형식이어야 합니다.
print(edge_indices_test.shape)   # (2, n) 형식이어야 합니다.

AttributeError: 'list' object has no attribute 'shape'

In [25]:
# 데이터 준비
train_data, val_data, test_data = prepare_data(trainset, testset, edge_indices_train, edge_indices_test)

# 데이터 모듈 설정
datamodule = GNNDataModule(train_data, val_data, test_data, batch_size=CFG['batch_size'])

# 모델 초기화
model = GCNModel(
    input_dim=CFG['input_dim'],
    hidden_dim=CFG['hidden_dim'],
    output_dim=CFG['output_dim'],
    num_layers=CFG['num_layers'],
    dropout=CFG['dropout']
)

# 학습
trainer = pl.Trainer(max_epochs=CFG['epochs'])
trainer.fit(model, datamodule)

train_x shape: (1366, 54), train_y shape: (1366,)
val_x shape: (586, 54), val_y shape: (586,)


TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
# 평가
rmse = evaluate_model(model, datamodule)
print(f'검증 데이터 RMSE: {rmse}')

# 테스트셋 예측
test_y_pred = model(test_data).detach().cpu().numpy()

# 제출 파일 생성
submit = pd.read_csv('sample_submission.csv')
submit['IC50_nM'] = pIC50_to_IC50(test_y_pred)
submit.to_csv('baseline_submit.csv', index=False)